# TLS 1.3 handshake overview
This is the basic TLS 1.3 handshake:

<img src="../notebooks/images/handshake_tls13.png" alt="Handshake TLS 1.3" width="400"/>

In [1]:
from scapy.all import *

In [2]:
record1_str = open('../notebooks/raw_data/tls_session_13/01_cli.raw').read()
record1 = TLS(record1_str)
record1.show()

###[ TLS ]### 
  type      = handshake
  version   = TLS 1.0
  len       = 512
  iv        = ''
  \msg       \
   |###[ TLS Handshake - Client Hello ]### 
   |  msgtype   = client_hello
   |  msglen    = 508
   |  version   = TLS 1.2
   |  gmt_unix_time= Mon, 08 Jan 2052 03:51:18 +0000 (2588298678)
   |  random_bytes= 50dcc81fed6f1fea635f15861574c0ed0bfb5778de7724fb927c5ef1
   |  sidlen    = 0
   |  sid       = ''
   |  cipherslen= 62
   |  ciphers   = [TLS_AES_128_GCM_SHA256, TLS_CHACHA20_POLY1305_SHA256, TLS_AES_256_GCM_SHA384, TLS_ECDHE_ECDSA_WITH_AES_128_GCM_SHA256, TLS_ECDHE_RSA_WITH_AES_128_GCM_SHA256, TLS_ECDHE_ECDSA_WITH_CHACHA20_POLY1305_SHA256, TLS_ECDHE_RSA_WITH_CHACHA20_POLY1305_SHA256, TLS_ECDHE_ECDSA_WITH_AES_256_CBC_SHA, TLS_ECDHE_ECDSA_WITH_AES_128_CBC_SHA, TLS_ECDHE_RSA_WITH_AES_128_CBC_SHA, TLS_ECDHE_ECDSA_WITH_AES_128_CBC_SHA256, TLS_ECDHE_RSA_WITH_AES_128_CBC_SHA256, TLS_ECDHE_RSA_WITH_AES_256_CBC_SHA, TLS_DHE_RSA_WITH_AES_128_GCM_SHA256, TLS_DHE_RSA_WITH_CHACHA20_P

In [3]:
# The PFS relies on the X25519 secret below being kept from observers, and deleted right after the key exchange
from cryptography.hazmat.primitives.asymmetric.x25519 import X25519PrivateKey
x25519_client_privkey = open('../notebooks/raw_data/tls_session_13/cli_key.raw').read()
privkey = X25519PrivateKey._from_private_bytes(x25519_client_privkey)
record1.tls_session.tls13_client_privshares["x25519"] = privkey

In [4]:
record2_str = open('../notebooks/raw_data/tls_session_13/02_srv.raw').read()
record2 = TLS(record2_str, tls_session=record1.tls_session.mirror())
record2.show()

###[ TLS ]### 
  type      = handshake
  version   = TLS 1.0
  len       = 82
  iv        = ''
  \msg       \
   |###[ TLS 1.3 Handshake - Server Hello ]### 
   |  msgtype   = server_hello
   |  msglen    = 78
   |  version   = TLS 1.3-d18
   |  random_bytes= 98e34364038683391cbec1039aa0fba2f496d8c8e6327151cc94bbc5ef739075
   |  cipher    = TLS_AES_128_GCM_SHA256
   |  extlen    = 40
   |  \ext       \
   |   |###[ TLS Extension - Key Share (for ServerHello) ]### 
   |   |  type      = key_share
   |   |  len       = 36
   |   |  \server_share\
   |   |   |###[ Key Share Entry ]### 
   |   |   |  group     = x25519
   |   |   |  kxlen     = 32
   |   |   |  key_exchange= '\xa2\x0e\xd1\xb7\xf2\xd9j\x7f\x12V\x8f\x0eF\x0b\xb0\xfc\x86\xdc\x8d\x1d\xb6\xc0}k\x10\xd4\xdct\xaa\xac\x92\x19'
  mac       = ''
  pad       = ''
  padlen    = None



In [5]:
record3_str = open('../notebooks/raw_data/tls_session_13/03_srv.raw').read()
record3 = TLS(record3_str, tls_session=record2.tls_session)
record3.show()

###[ TLS 1.3 ]### 
  type      = application_data
  version   = TLS 1.0
  len       = 668    [deciphered_len= 652]
  \inner     \
   |###[ TLS Inner Plaintext ]### 
   |  \msg       \
   |   |###[ TLS 1.3 Handshake - Encrypted Extensions ]### 
   |   |  msgtype   = encrypted_extensions
   |   |  msglen    = 30
   |   |  extlen    = 28
   |   |  \ext       \
   |   |   |###[ TLS Extension - Supported Groups ]### 
   |   |   |  type      = supported_groups
   |   |   |  len       = 20
   |   |   |  groupslen = 18
   |   |   |  groups    = [x25519, secp256r1, secp384r1, secp521r1, ffdhe2048, ffdhe3072, ffdhe4096, ffdhe6144, ffdhe8192]
   |   |   |###[ TLS Extension - Server Name ]### 
   |   |   |  type      = server_name
   |   |   |  len       = 0
   |   |   |  servernameslen= None
   |   |   |  servernames= []
   |   |###[ TLS 1.3 Handshake - Certificate ]### 
   |   |  msgtype   = certificate
   |   |  msglen    = 441
   |   |  cert_req_ctxt_len= 0
   |   |  cert_req_ctxt= ''
   |   |

In [6]:
record4_str = open('../notebooks/raw_data/tls_session_13/04_cli.raw').read()
record4 = TLS(record4_str, tls_session=record3.tls_session.mirror())
record4.show()

###[ TLS 1.3 ]### 
  type      = application_data
  version   = TLS 1.0
  len       = 53    [deciphered_len= 37]
  \inner     \
   |###[ TLS Inner Plaintext ]### 
   |  \msg       \
   |   |###[ TLS Handshake - Finished ]### 
   |   |  msgtype   = finished
   |   |  msglen    = 32
   |   |  vdata     = '\x1a^\xb0\xba_\x92\xf3N\xd0\x05\x9dd\xce\xdd*} \x8f%\xf0\x0e(\x13\x81\x17\xfb9t\xd4\x15wj'
   |  type      = handshake
   |  pad       = ''
  auth_tag  = '\xec\xfc\x96\x96$\xd6\x8e\x19\xd8\x8c>g\xcc\xb4\x8b\xdf'



In [7]:
record5_str = open('../notebooks/raw_data/tls_session_13/05_srv.raw').read()
record5 = TLS(record5_str, tls_session=record4.tls_session.mirror())
record5.show()

###[ TLS 1.3 ]### 
  type      = application_data
  version   = TLS 1.0
  len       = 187    [deciphered_len= 171]
  \inner     \
   |###[ TLS Inner Plaintext ]### 
   |  \msg       \
   |   |###[ TLS Handshake - New Session Ticket ]### 
   |   |  msgtype   = session_ticket
   |   |  msglen    = 166
   |   |  ticket_lifetime= 172800
   |   |  ticket_age_add= 1358609079
   |   |  ticketlen = 146
   |   |  ticket    = 'NSS!\xe5\xc1\x02\xaa$\x98R\xac\x81\xf0\x80\xbcb\xbd^ij\r\x8a!0\xe9\x80\xe3{\x905\xaa\x00P@:\tE\x1cI\x7f\x08%`\x9b\xac\x97lY\xf2\xd5?\x15\x90Q\xd2\xfc_\xcfK\xd6\x8a\xe4\x88m\xd9\xa0QD\xd0|Z&F\x17~\x94\x01_vN\xdd^\xcf;~\xa4\x04/)\xa6"P\x92\xb1\x1b\xa8\xec \xe1|\\o\xff\x9a8\xc5l\xf27;+\xd58`\xe5\xb0\xb9\x83\xe8*\xa2\xaer\xcb"[\x9diQg\xed\tc\xcc\xea\xbf\xa1\t\xe1\xb2\xe5\x10O\xec4'
   |   |  extlen    = 8
   |   |  \ext       \
   |   |   |###[ TLS Extension - Ticket Early Data Info ]### 
   |   |   |  type      = ticket_early_data_info
   |   |   |  len       = 4
   |   |   |

In [8]:
record6_str = open('../notebooks/raw_data/tls_session_13/06_cli.raw').read()
record6 = TLS(record6_str, tls_session=record5.tls_session.mirror())
record6.show()

###[ TLS 1.3 ]### 
  type      = application_data
  version   = TLS 1.0
  len       = 67    [deciphered_len= 51]
  \inner     \
   |###[ TLS Inner Plaintext ]### 
   |  \msg       \
   |   |###[ TLS Application Data ]### 
   |   |  data      = '\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\x0c\r\x0e\x0f\x10\x11\x12\x13\x14\x15\x16\x17\x18\x19\x1a\x1b\x1c\x1d\x1e\x1f !"#$%&\'()*+,-./01'
   |  type      = application_data
   |  pad       = ''
  auth_tag  = '\x0c\xc3\xe7`J\xe6a\xf0\xaf\xa2\xf7u\xc3f\x88g'



## Observations sur TLS 1.3
* Certificat désormais chiffré...
* ...mais pas le Server Name dans le ClientHello
* Risques du mode 0-RTT